<a href="https://colab.research.google.com/github/justinchan114/landsat_clustering/blob/main/Landuse_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [29]:
#Define coordinate of study, an area near Metro Vancouver
coords = [[-122.4,49.0],
          [-122.0,49.0],
          [-122.0,49.2],
          [-122.4,49.2],
          [-122.4,49.0]]

van = ee.Geometry.Polygon(coords)

#Get the landsat 8 image set(USGS Landsat 8 Collection 2 Tier 1 and Real-Time data Raw Scenes)
input = ee.ImageCollection("LANDSAT/LC08/C02/T1_RT")\
    .filterBounds(van)\
    .select(["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B10", "B11"])

#Computes a Landsat TOA composite from a collection of raw Landsat scenes
#Making image composite to ensure the image could cover the area of study with least clouds
#The way using .sort("CLOUD_COVER").first() will only return one image and does not cover the whole area of study
landsat8 = ee.Algorithms.Landsat.simpleComposite(
    collection = input.filterDate('2018-05-01', '2018-08-31'),
    asFloat=True)

#Making the training dataset
training = landsat8.sample(region=van, scale=30, numPixels=5000)

#Instantiate the clusterer and train it
clusterer = ee.Clusterer.wekaKMeans(10).train(training)

#Cluster the image set(landsat8) using the trained clusterer
result = landsat8.clip(van).cluster(clusterer)

In [30]:
import folium

In [21]:
# Define a function to add a Earth Engine layer to folium
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles = map_id_dict['tile_fetcher'].url_format,
      attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name = name,
      overlay = True,
      control = True
  ).add_to(self)

In [32]:
# Define the visualizer parameters
vis_params = {'min': 0, 'max': 9, 'palette': ['red', 'blue', 'green', 'yellow', 'purple', 'orange', 'pink', 'brown', 'black', 'white']}


#Parameters for displaying rgb as base map
rgb_params = {'bands' : ["B4","B3","B2"],
              'gamma': 1,
              'max': 0.19,
              'min': 0.02}

# Create a folium map
m = folium.Map(location=[49.1, -122.2], zoom_start=12)

add_ee_layer(m, landsat8.clip(van), rgb_params, 'rgb')
add_ee_layer(m, result.clip(van), vis_params, 'clusters')

m.add_child(folium.LayerControl())

# Display the map
m
